# 5 Implied Volatility

This notebook demonstrates implied volatility calculations using QuantLib, comparing Black-Scholes and Heston models.
We'll calculate implied volatilities for options and analyze their deltas.

In [1]:
import sys
import os
import pandas as pd

# Display current working directory and update Python path
current_path = os.getcwd()
print("Current Path:", current_path)
current_path = current_path.replace('/samples', '')
print(sys.path)
sys.path.append(current_path)
print(sys.path)


Current Path: /Users/yang/PycharmProjects/Option/samples
['/Users/yang/PycharmProjects/Option/samples', '/Users/yang/PycharmProjects/Option', '/Applications/PyCharm.app/Contents/plugins/python-ce/helpers/pydev', '/Applications/PyCharm.app/Contents/plugins/python-ce/helpers/jupyter_debug', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa']
['/Users/yang/PycharmProjects/Option/samples', '/Users/yang/PycharmProjects/Option', '/Applications/PyCharm.app/Contents/plugins/python-ce/helpers/pydev', '/Applications/PyCharm.app/Contents/plugins/python-ce/helpers/jupyter_debug', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa', '/Users/yang/PycharmProjects/Option']


In [2]:
# Import required libraries for quantitative analysis
import numpy as np
import QuantLib as ql
import matplotlib.pyplot as plt
import time

from src.utils import plot_fig
from src.QlCalendar import QlCalendar
from src.QlStocks import QlStocks
from src.QlEuropeanOptions import QlEuropeanOptions


In [3]:
# Initialize model parameters for option pricing
# S0: Initial stock price
# strike: Option strike price
# r: Risk-free rate
# sigma: Volatility (Black-Scholes)
# T: Time to maturity
# steps: Number of time steps

r = 0.03  # Risk-free interest rate
S0 = 100  # Initial stock price
strike = 100  # Strike price
sigma = 0.13  # Black-Scholes volatility

# Heston model parameters
V0 = 0.04  # Initial volatility
kappa = 2.0  # Mean reversion speed
theta = 0.04  # Long-term volatility mean
sigma_heston = 0.3  # Volatility of volatility
rho = -0.7  # Price-volatility correlation

T = 1  # Time to maturity (1 year)
optionType = "call"  # Option type
steps = 252  # Number of time steps (trading days)


In [4]:
# Set up QuantLib calendar and calculate maturity date
# Uses business days for financial calculations
start_date = ql.Date(3, 1, 2023)
ql_calendar = QlCalendar(
    init_date=start_date,
    init_risk_free_rate=r,
)

end_date = ql_calendar.cal_date_advance(times=steps, time_unit='days')


In [5]:
# Verify date calculations between start and maturity dates
print("Maturity Date:", end_date)
print("Days between start and end:", ql_calendar.calendar.businessDaysBetween(start_date, end_date))


Maturity Date: January 10th, 2024
Days between start and end: 252


In [6]:
# Initialize Heston model and Black-Scholes model with same price quotes
ql_stocks = QlStocks(ql_calendar)
ql_stocks.add_heston(codes='STCKHM',
                     stock_prices=S0,
                     v0=V0,
                     kappa=kappa,
                     theta=theta,
                     rho=rho,
                     sigma=sigma_heston,
                     dividend_rates=0.0,
                     )

# Create Heston model option
stock_heston = ql_stocks.stock('STCKHM')
option_heston = stock_heston.set_european_option(
    option_type='call',
    strike_price=strike,
    maturity_date=end_date
)
option_heston.analytic_engine()


In [7]:
# Create corresponding Black-Scholes model with same price quotes
price_quotes = stock_heston.get_price_quote()
ql_stocks.add_black_scholes('STCKBS', price_quotes, sigma=sigma)
stock_bs = ql_stocks.stock('STCKBS')
option_bs = stock_bs.set_european_option(
    option_type='call',
    strike_price=strike,
    maturity_date=end_date
)
option_bs.analytic_engine()


In [8]:
# Calculate implied volatility when stock price changes to 110
stock_bs.set_today_price(110)

npv_heston = option_heston.NPV()
delta_heston = option_heston.delta_numerical()

# Calculate implied volatility matching Heston model's NPV
implied_vol = option_bs.impliedVolatility(npv_heston)
print(f"Implied Volatility: {implied_vol:.4f}")

# Update BS model with implied volatility
stock_bs.set_volatility(implied_vol)

# Compare NPV and delta between models
npv_bs = option_bs.NPV()
delta_bs = option_bs.delta()
print(f'Heston model vs BS model comparison:')
print(f'Heston NPV: {npv_heston} | BS NPV: {npv_bs}')
print(f'Heston Delta: {delta_bs} | BS Delta: {delta_bs}')


Implied Volatility: 0.2092
Heston model vs BS model comparison:
Heston NPV: 16.703688763927715 | BS NPV: 16.702881510646854
Heston Delta: 0.7585627613457159 | BS Delta: 0.7585627613457159


## Calculating Multiple Implied Volatilities

This section demonstrates calculating implied volatilities for multiple option prices and stock prices.

In [9]:
# Calculate implied volatilities for multiple option prices and stock prices
implied_vols = option_bs.impliedVolatility_multi(
    [10, 11, 12, 13], stock_prices=[100, 101, 102, 103]
)
print(f"Implied Volatilities: {implied_vols}")


Implied Volatilities: [0.21239656310313196, 0.22246947198998218, 0.23215302172892247, 0.2415292919518508]


## Calculating Implied Volatility with Delta

This section shows how to simultaneously calculate implied volatility and its corresponding delta.

In [10]:
# Calculate both implied volatility and delta for given option price
vol, delta = option_bs.impliedVolatility_and_delta(npv_heston)
print(f"Volatility: {vol}, Delta: {delta}")


Volatility: 0.33656447390354477, Delta: 0.6356554025281472


## Multiple Implied Volatilities with Deltas

This section demonstrates batch calculation of implied volatilities and deltas for multiple option prices.

In [11]:
# Calculate multiple implied volatilities and deltas
vols, deltas = option_bs.impliedVolatility_and_delta_multi(
    [npv_heston, 14., 15.], stock_prices=[110., 110., 105.]
)
print(f"Volatilities: {vols}")
print(f"Deltas: {deltas}")

Volatilities: [0.20918727 0.12092521 0.25931542]
Deltas: [0.75856276 0.86263141 0.66787744]


# Implied Volatility Calculation Analysis with different T


## T = 1 month

In [12]:
# Set the current date in QuantLib's calendar to December 11th, 2023
ql_calendar.set_today(ql.Date(11,12,2023))

# Print formatted date information showing both today's date and the end date
print(f'[Date Info] Today: {ql_calendar.today()}, End Date: {end_date}')

# Set the current stock price to $130 for Black-Scholes calculations
stock_bs.set_today_price(130)

# Set the volatility parameter (sigma) for the option pricing model
option_bs.stock_vol_quote.setValue(sigma)

# Print volatility and NPV (Net Present Value) with explanations
print(f'[Pricing Parameters] Volatility (σ): {option_bs.stock_vol_quote.value():.4f} , Option NPV: ${option_bs.NPV():.2f}')

# Calculate and print implied volatility based on current NPV
print(f'[Implied Volatility] Calculated σ: {option_bs.impliedVolatility(option_bs.NPV()):.4f} (derived from current option price)')

[Date Info] Today: December 11th, 2023, End Date: January 10th, 2024
[Pricing Parameters] Volatility (σ): 0.1300 , Option NPV: $30.25
[Implied Volatility] Calculated σ: 0.1300 (derived from current option price)


## T = 1 day


In [29]:
# Set the current date in QuantLib's calendar to December 11th, 2023
ql_calendar.set_today(ql.Date(9,1,2024))

# Print formatted date information showing both today's date and the end date
print(f'[Date Info] Today: {ql_calendar.today()}, End Date: {end_date}')

# Set the current stock price to $130 for Black-Scholes calculations
stock_bs.set_today_price(110)

# Set the volatility parameter (sigma) for the option pricing model
option_bs.stock_vol_quote.setValue(sigma)

# Print volatility and NPV (Net Present Value) with explanations
print(f'[Pricing Parameters] Volatility (σ): {option_bs.stock_vol_quote.value():.4f} , Option NPV: ${option_bs.NPV():.2f}')

# Calculate and print implied volatility based on current NPV
print(f'[Implied Volatility] Calculated σ: {option_bs.impliedVolatility(option_bs.NPV()):.4f} (derived from current option price)')

[Date Info] Today: January 9th, 2024, End Date: January 10th, 2024
[Pricing Parameters] Volatility (σ): 0.1300 , Option NPV: $10.01
[Implied Volatility] Calculated σ: 0.0000 (derived from current option price)


In [32]:
option_bs.impliedVolatility(10.1)

1.0400247170358274

#